In [2]:
!pip install stop-words
!pip install nltk

  Preparing metadata (setup.py) ... done
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=f4f52fe786a0267bacb78bcf77eec11fcd0c87581f8d5e56b5b18af73a804963
  Stored in directory: /home/jupyter/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words
     |████████████████████████████████| 1.5 MB 8.6 MB/s            


In [3]:
import re
import nltk
import string
import unicodedata
import pandas as pd
from stop_words import get_stop_words

#from sklearn.metrics.pairwise import  cosine_similarity
#from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
re_tab_pattern       = r"(\t)"
re_return_pattern    = r"(\r)"
re_comma_pattern     = r"(,{2,})"
re_space_pattern     = r"(\s{1,})"
re_linefeed_pattern  = r"(\n)"

re_tab_obj           = re.compile(re_tab_pattern)
re_return_obj        = re.compile(re_return_pattern)
re_comma_obj         = re.compile(re_comma_pattern)
re_space_obj         = re.compile(re_space_pattern)
re_linefeed_obj      = re.compile(re_linefeed_pattern)

stop_ptBr = set(nltk.corpus.stopwords.words('portuguese'))
punctuation = list(string.punctuation)
stop_ptBr.update(punctuation)

#stemmer_ptBr = nltk.stem.snowball.SnowballStemmer("portuguese")

In [5]:
def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")
    return str(text)

def remove_dot(text):
    return re.sub(r'\.(?!\d)', '', text)

def remove_numbers(text):
    return re.sub(r'[0-9]', '', text)

def all_Regex_transformations(text, has_space=True):
    final_text = text

    if has_space is False:
        final_text = final_text.replace(' ' , '_')

    final_text = strip_accents(final_text)
    final_text = remove_dot(final_text)
    final_text = remove_numbers(final_text)
    final_text = re.sub(re_comma_obj     , ''  , final_text)
    final_text = final_text.replace('\t' , ' ')
    final_text = final_text.replace('\n' , ' ')
    final_text = final_text.replace('\r' , ' ')
    final_text = final_text.replace('\\' , ' ')
    final_text = final_text.replace(','  , ' ')
    final_text = final_text.replace('.'  , ' ')
    final_text = final_text.replace('-'  , ' ')
    final_text = final_text.replace('/'  , ' ')    
    final_text = final_text.replace('"'  , ' ')
    final_text = final_text.replace(')'  , ' ')
    final_text = final_text.replace('('  , ' ')
    final_text = final_text.replace('!'  , ' ')
    final_text = final_text.replace('?'  , ' ')
    final_text = re.sub(re_space_obj     , ' '  , final_text)

    return final_text.lower()

In [22]:
df = pd.read_csv('../data/raw/bq-results-20211013.csv')


df['sinopse_prep']  = df.apply(lambda x: all_Regex_transformations(text=x['synopsis']) , axis=1 )
df['sinopse_token'] = df['sinopse_prep'].apply(nltk.tokenize.word_tokenize) 
df['sinopse_token_stop'] = df['sinopse_token'].apply(lambda x: [item for item in x if item not in stop_ptBr])
df['word len'] = df['sinopse_token_stop'].apply(lambda x: len(x)>10)
df = df[df['word len']==True]
df['sent_StopW'] = df['sinopse_token_stop'].apply(lambda x: ' '.join(x))


TypeError: normalize() argument 2 must be str, not float